In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from modelArh import CarGameAgentDoubleMaybe,CombinedCarGameAgentMaybe  # Your custom model
from balance_it import load_and_balance_for_double,load_and_balance_for_double_maybe,load_only
from GATrainer import get_last_gen
from Functions import load_all_levels
from GATrainer import env_fn
import numpy as np
import time
import Functions
import pygame


/home/pfe-admin/komporess/Jedinica - Copy/clean/.venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:


# === Parameters ===
recordings_folder = "/home/pfe-admin/komporess/Jedinica - Copy/clean/clean-codes/recordings_gb"
batch_size = 64
epochs = 100
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gen = 0

ray_number = 7
parametri = 6
stanja = 4
n_inputs = parametri + stanja + 2 * ray_number

In [3]:


# # === Load and unpack data ===
# gas_brake, left_right = load_and_balance_for_double(recordings_folder)

# gb_states = [s for s, _ in gas_brake]
# gb_actions = [a for _, a in gas_brake]

# lr_states = [s for s, _ in left_right]
# lr_actions = [a for _, a in left_right]

# print(f"Loaded {len(gb_states)} gas/brake samples after balancing.")
# print(f"Loaded {len(lr_states)} left/right samples after balancing.")


# === Load and unpack data ===
import random

gas_brake, left_right = load_only(recordings_folder)

gb_states = [s for s, _ in gas_brake]
lr_states = [s for s, _ in left_right]

gb_actions = []
lr_actions = []

# Counters for balancing
gb_counts = [0, 0, 0]  # Gas, Brake, Neither
lr_counts = [0, 0, 0]  # Left, Right, Neither

# Process Gas/Brake actions
for _, action in gas_brake:
    if action[0]:
        gb_actions.append(0)
        gb_counts[0] += 1
    elif action[1]:
        gb_actions.append(1)
        gb_counts[1] += 1
    else:
        gb_actions.append(2)
        gb_counts[2] += 1

# Process Left/Right actions
for _, action in left_right:
    if action[0]:
        lr_actions.append(0)
        lr_counts[0] += 1
    elif action[1]:
        lr_actions.append(1)
        lr_counts[1] += 1
    else:
        lr_actions.append(2)
        lr_counts[2] += 1

print(f"Initial GB Counts: {gb_counts}")
print(f"Initial LR Counts: {lr_counts}")
# print(gb_actions[:10])
# --- Boost "Neither" Class ---

# def boost_class(states, actions, target_class, desired_total):
#     combined = list(zip(states, actions))
#     current_count = actions.count(target_class)

#     while current_count < desired_total:
#         for s, a in combined:
#             if a == target_class:
#                 states.append(s)
#                 actions.append(a)
#                 current_count += 1
#                 if current_count >= desired_total:
#                     break

# # Desired balancing: max(all classes) count for each class
# max_gb = max(gb_counts)
# max_lr = max(lr_counts)




# for i in range(3):
#     boost_class(lr_states, lr_actions, i, max_lr)
    

# Optional: shuffle after boosting
# gb_states = [s for s, _ in gas_brake]
# gb_actions = [a for _, a in gas_brake]




if lr_counts[0] > lr_counts[1]:
    diff = lr_counts[0] - lr_counts[1]
    minority_samples = [sample for sample in left_right if sample[1][1]]
    for _ in range(diff):
        left_right.append(random.choice(minority_samples))





combined_gb = list(zip(gb_states, gb_actions))
random.shuffle(combined_gb)
gb_states, gb_actions = zip(*combined_gb)


combined_lr = list(zip(lr_states, lr_actions))
random.shuffle(combined_lr)
lr_states, lr_actions = zip(*combined_lr)

# print(f"After Balancing - GB: {gb_actions.count(0)} Gas, {gb_actions.count(1)} Brake, {gb_actions.count(2)} Neither")
# print(f"After Balancing - LR: {lr_actions.count(0)} Left, {lr_actions.count(1)} Right, {lr_actions.count(2)} Neither")


print(f"Loaded {len(gb_states)} gas/brake samples after balancing.")
print(f"Loaded {len(lr_states)} left/right samples after balancing.")


bg_number=[0,0,0]

for action in gb_actions:
    bg_number[action]+=1

print(bg_number)

gb_actions=list(gb_actions)
gb_states=list(gb_states)

# print(gb_actions)


lr_number=[0,0,0]   

for action in lr_actions:
    lr_number[action]+=1

print(lr_number)

lr_actions=list(lr_actions)
lr_states=list(lr_states)





Initial GB Counts: [13320, 2607, 279]
Initial LR Counts: [2380, 2831, 10995]
Loaded 16206 gas/brake samples after balancing.
Loaded 16206 left/right samples after balancing.
[13320, 2607, 279]
[2380, 2831, 10995]


In [4]:
from ClassesML2 import vis_single_state_cv2
z=random.randint(0,100)
# for i in range(z+i,z+i+100):
#     print(gb_states[i],end="\n")

b=gb_states[z]
inters=b[-14:]
for i in range(len(inters)):
    inters[i]=inters[i]=(2*inters[i]-1)*2000
print(gb_states[z])
print(inters)
# import cv2
# vis_single_state_cv2(7,b)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

[0.05, 0.2, 0.2, 0.25, 0.041666666666666664, 0.15, 0.6652051886367969, 0.028081314581123673, 0.33113596232067566, 0.5734871162431724, 0.514371791173464, 0.5161466329787587, 0.526532699348531, 0.6384756905436239, 0.5971904317686761, 0.5556519232839997, 0.5468379350316607, 0.5144623305668569, 0.5155973849007843, 0.523865001724249, 0.6371437285607235, 0.6021914818005889, 0.5585918873572647, 0.5471354371549397]
[57.48716469385595, 64.58653191503494, 106.1307973941239, 553.9027621744954, 388.7617270747046, 222.60769313599883, 187.35174012664268, 57.84932226742745, 62.38953960313731, 95.46000689699596, 548.5749142428941, 408.7659272023556, 234.3675494290589, 188.54174861975892]


In [5]:
# cv2.destroyAllWindows()